In [ ]:
import re
import json
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
def iter_json(text: str):
    """
    Intenta cargar el JSON completo, incluso si está todo en una línea.
    Si falla, busca bloques 'Event_*' dentro del texto.
    """
    try:
        obj = json.loads(text)
        if isinstance(obj, dict):
            yield obj
            return
    except Exception:
        pass

    # Si llega aquí, el JSON completo no se pudo parsear.
    # Extraemos los sub-bloques "Event_x" usando regex:
    pattern = r'"Event_\d+"\s*:\s*\{.*?\}(?=,"Event_\d+"|}})'
    matches = re.findall(pattern, text)
    if matches:
        # reconstruimos un objeto "Output"
        yield {"Output": json.loads("{" + ",".join(matches) + "}")}

def get_in(d, keys, default=None):
    cur = d
    for k in keys:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return default
    return cur

def events_to_dataframe_2(text: str):
    """Convierte el contenido JSON de los eventos en un DataFrame con todas las columnas disponibles."""
    records = []

    for rec in iter_json(text):
        out = rec.get("Output", {})
        for evname, ev in out.items():
            flux = ev.get("InputFlux", {})
            det = ev.get("Detector_0", {})
            opt = det.get("OptDevice_0", {})

            row = {
                "Event": evname,
                "ID": flux.get("ID"),
                "Position_x": flux.get("Position", [None]*3)[0],
                "Position_y": flux.get("Position", [None]*3)[1],
                "Position_z": flux.get("Position", [None]*3)[2],
                "Momentum_x": flux.get("Momentum", [None]*3)[0],
                "Momentum_y": flux.get("Momentum", [None]*3)[1],
                "Momentum_z": flux.get("Momentum", [None]*3)[2],
            }

            # Agregar campos directos de Detector_0
            for k, v in det.items():
                if k != "OptDevice_0" and not isinstance(v, dict):
                    row[k] = v

            # Agregar todas las categorías dentro de OptDevice_0
            for k, v in opt.items():
                if isinstance(v, list):
                    row[k] = v  # guardamos la lista completa
                else:
                    row[k] = v

            records.append(row)

    return pd.DataFrame(records)

In [ ]:
def Charge_Histogram_All(df,df_2, df_3,cluster_col='Cluster', bin_width=7):
#def Charge_Histogram_All(df,cluster_col='Cluster',bin_width=7):
    """
    Generates an histogram from the column 'Charge':
    - Global all data
    - Splits the data according to each cluster.

    Parameters:
    - df: DataFrame with the column 'Charge' could be cluster_col (ej: 0,1,2)
    - cluster_col: name of the cluster
    - bin_width: the width of the bins
    """
    plt.figure(figsize=(15, 8))

    # --- bins compartidos para todos ---

    min_val, max_val = df['Charge'].min(), df['Charge'].max()
    bins = np.arange(min_val, max_val + bin_width, bin_width)

    # --- histograma global ---
    counts, bin_edges = np.histogram(df['Charge'], bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.semilogy(bin_centers, counts, color='red', lw=3,alpha=0.6,label="Total", zorder=5)

    counts_2, bin_edges_2 = np.histogram(df_2['Charge'], bins=bins)
    bin_centers_2 = (bin_edges_2[:-1] + bin_edges_2[1:]) / 2
    plt.semilogy(bin_centers_2, counts_2, color='black', lw=3,alpha=0.8,linestyle='--',label="Muons", zorder=5)

    counts_3, bin_edges_3 = np.histogram(df_3['Charge'], bins=bins)
    bin_centers_3 = (bin_edges_3[:-1] + bin_edges_3[1:]) / 2
    plt.semilogy(bin_centers_3, counts_3, color='#BF00BF', lw=3,alpha=0.8,linestyle='--', label="Electrons", zorder=5)

    # --- histogramas por cluster ---
    clusters = sorted(df[cluster_col].unique())
    for c in clusters:
        subset = df[df[cluster_col] == c]['Charge'].to_numpy()
        counts, bin_edges = np.histogram(subset, bins=bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        plt.semilogy(bin_centers, counts, label=f"Cluster {c}", alpha=0.9, zorder=4)

    plt.ylabel("Number of Counts", fontsize=20)
    plt.xlabel("ADC", fontsize=20)
    #plt.title("Charge Histogram", fontsize=20)
    plt.grid(True)
    plt.legend(fontsize='20')
    plt.show()

In [ ]:
def analizar_evento(tiempos_pe):
    """
    Analiza un evento a partir de los tiempos de llegada de los fotoelectrones.

    Parámetros:
    -----------
    tiempos_pe : list o array
        Tiempos de llegada de los fotoelectrones (en segundos).
    bins : int
        Número de bins para el histograma.

    Retorna:
    --------
    dict con:
        - 'Total_PEs_Deposited': Total de PEs del evento
        - 'Pico': Máximo número de PEs en un bin
        - 'Tiempo_90': Tiempo que tarda en depositar el 90% de los PEs (ns)
        - 'Duration': Duración del pulso (ns)
        - 'bin_centers': Centros de bins (ns)
        - 'counts': Número de PEs por bin
    """

    # Convertir tiempos a ns
    tiempos_ns = np.array(tiempos_pe) * 1e9

    # Mantener 8 ns siempre
    # Asegurarse de que `bins` sea al menos 1 para evitar errores con np.histogram
    time_range = tiempos_ns.max() - tiempos_ns.min()
    bins = int(time_range/8)
    if bins <= 0:
      print(bins)
      bins = 1

    # Histograma
    counts, bin_edges = np.histogram(tiempos_ns, bins=bins)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Total de PEs
    Total_PEs_Deposited = counts.sum()

    # Pico del pulso
    Pico = counts.max()

    # DataFrame para la acumulada
    df = pd.DataFrame({
        'tiempos': bin_centers,
        'Num_fotoelectrones': counts
    })
    df['frecuencia_acumulada'] = df['Num_fotoelectrones'].cumsum()

    # Tiempo para depositar 90% de la señal
    # Si hay un solo bin, o pocos datos, puede que el 90% sea el primer bin center
    if not df.empty and Total_PEs_Deposited > 0:
        tiempo_90_signal = df[df['frecuencia_acumulada'] >= Total_PEs_Deposited*0.9]['tiempos'].iloc[0]
    else:
        tiempo_90_signal = None # O un valor por defecto si no hay PEs o datos

    # Duración del pulso
    if len(bin_centers) > 1:
        duration = bin_centers.max() - bin_centers.min()
    elif len(bin_centers) == 1:
        duration = 0.0 # O el ancho del bin si se considera relevante
    else:
        duration = None

    return {
        'Total_PEs_Deposited': Total_PEs_Deposited,
        'Pico': Pico,
        'Tiempo_90': tiempo_90_signal,
        'Duration': duration,
        'bin_centers': bin_centers,
        'counts': counts
    }

In [ ]:
def Add_Time_Features(df, column='PETimeDistribution'):
    """
    Añade al DataFrame las características temporales de cada evento.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con la columna de tiempos de PEs por evento.
    column : str
        Nombre de la columna con los tiempos de PEs.
    bins : int
        Número de bins para el histograma.

    Retorna:
    --------
    df : pd.DataFrame
        DataFrame con columnas adicionales:
        - 'Total_PEs_Deposited'
        - 'Time_90_PEs_Deposited'
        - 'Pulse_Duration'
    """

    all_Total_PEs = []
    all_Peak = []
    all_Time_90 = []
    all_Duration = []

    for i in range(len(df[column])):
        tiempos_pe = df[column][i]
        resultado = analizar_evento(tiempos_pe)
        all_Total_PEs.append(resultado['Total_PEs_Deposited'])
        all_Peak.append(resultado['Pico'])
        all_Time_90.append(resultado['Tiempo_90'])
        all_Duration.append(resultado['Duration'])

    df['Total_PEs_Deposited'] = all_Total_PEs
    df['Pico'] = all_Peak
    df['Time_90_PEs_Deposited'] = all_Time_90
    df['Pulse_Duration'] = all_Duration

    return df

In [ ]:
def particles(df,ID):
  df_particles = df[(df['ID'] == ID[0]) | (df['ID'] == ID[1])]
  df_particles = df_particles[df_particles['PETimeDistribution'].astype(bool) & df_particles['PETimeDistribution'].apply(lambda arr: len(arr) > 9)]
  df_particles = df_particles[['Event','ID','Position_x','Position_y','Position_z','Momentum_x','Momentum_y','Momentum_z','EnergyDeposit','Charge','PETimeDistribution']]
  df_particles.reset_index(drop=True, inplace=True)
  return df_particles

In [ ]:
with gzip.open('/content/drive/MyDrive/Rayos Cósmicos/Meiga/chiapas_parte_1.2.json.gz', 'rt', encoding='utf-8') as f:
  text = f.read()
df_Chiapas = events_to_dataframe_2(text)

In [ ]:
# Time has chaged to nanoseconds
ID = [13,-13]
df_Chiapas_muones = particles(df_Chiapas,ID)
ID = [11,-11]
df_Chiapas_electrones = particles(df_Chiapas,ID)

In [ ]:
Add_Time_Features(df_Chiapas_muones)

In [ ]:
df_muons_process =  df_Chiapas_muones[['ID','Total_PEs_Deposited','Pico','Time_90_PEs_Deposited','Pulse_Duration','Charge']]
df_electrones_process =  df_Chiapas_electrones[['ID','Total_PEs_Deposited','Pico','Time_90_PEs_Deposited','Pulse_Duration','Charge']]

In [ ]:
df_all = pd.concat([df_muons_process,df_electrones_process])

In [ ]:
df_mezclado = df_all.sample(frac=0.5).reset_index(drop=True)
df_mezclado['ID'] = abs(df_mezclado['ID'])
#df_test = df_mezclado.head(100000)
#df_test = df_mezclado

In [ ]:
df_mezclado

# **OPTICS para las simulaciones:
El primer paso consiste en utilizar PCA. Esto reduce las dimensiones a dos.

In [ ]:
#df_clean = df_test.copy()
#features_clean = df_clean[['Total_PEs_Deposited','Pico','Time_90_PEs_Deposited','Pulse_Duration']]

In [ ]:
from scipy.stats import zscore

features = df_test[['Total_PEs_Deposited','Pico','Time_90_PEs_Deposited','Pulse_Duration']]

# Z-score absoluto
Z = np.abs(zscore(features))

# Máscara: fila es outlier si alguna columna tiene Z > 3
outlier_mask = (Z >2).any(axis=1)

# DataFrame limpio
df_clean = df_test[~outlier_mask].reset_index(drop=True)

# Porcentaje de filas eliminadas
porcentaje_perdida = 100 * outlier_mask.sum() / len(df_test)
print(f"Porcentaje de pérdida por outliers: {porcentaje_perdida:.2f}%")

In [ ]:
print('Promedio num PE:',df_clean['Total_PEs_Deposited'].mean())
print('Promedio Pico:',df_clean['Pico'].mean())
print('Promedio Tiempo_90:',df_clean['Time_90_PEs_Deposited'].mean())
print('Promedio Duracion_pulso:',df_clean['Pulse_Duration'].mean())

In [ ]:
df_clean = df_clean[['Total_PEs_Deposited','Pico','Time_90_PEs_Deposited','Pulse_Duration']]

In [ ]:
# Escalado y PCA
X_scaled = StandardScaler().fit_transform(df_clean)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# IDs de partículas
#particles_ID = df_clean['ID']

# Colores según ID
#cmap = plt.cm.Dark2
#norm = plt.Normalize(vmin=min(particles_ID), vmax=max(particles_ID))

# Leyenda personalizada
#id_labels = {11: 'Electrons (ID=11)', 13: 'Muons (ID=13)'}
#unique_ids = np.unique(particles_ID)
#handles = [plt.Line2D([0], [0], marker='o', color='w',
#                      markerfacecolor=cmap(norm(uid)),
#                     markersize=8, label=id_labels.get(uid, f'ID {uid}'))
#           for uid in unique_ids]

# Gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# PCA1 vs PCA2
axes[0].scatter(X_pca[:, 0], X_pca[:, 1], s=20, alpha=0.6)
axes[0].set_xlabel("PCA1")
axes[0].set_ylabel("PCA2")
#axes[0].set_title("Proyección PCA1 vs PCA2")

# PCA1 vs PCA3
axes[1].scatter(X_pca[:, 0], X_pca[:, 2], s=20, alpha=0.6)
axes[1].set_xlabel("PCA1")
axes[1].set_ylabel("PCA3")
#axes[1].set_title("PCA1 vs PCA3")

# PCA2 vs PCA3
axes[2].scatter(X_pca[:, 1], X_pca[:, 2], s=20, alpha=0.6)
axes[2].set_xlabel("PCA2")
axes[2].set_ylabel("PCA3")
#axes[2].set_title("Proyección PCA2 vs PCA3")

# Leyenda y diseño
#fig.legend(handles=handles, title="Particle", loc="upper right")
plt.tight_layout()
plt.show()



In [ ]:
from sklearn.cluster import OPTICS
import matplotlib.pyplot as plt

optics = OPTICS(min_samples=2000, xi=0.1, max_eps=2, min_cluster_size=5000)
labels = optics.fit_predict(X_pca)

# --- Configuración de color ---
unique_labels = np.unique(labels)
cmap = plt.cm.tab10
norm = plt.Normalize(vmin=min(unique_labels), vmax=max(unique_labels))

# --- Figura con tres proyecciones ---
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# PCA1 vs PCA2
axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap=cmap, s=10, alpha=0.6)
axes[0].set_xlabel("PCA1")
axes[0].set_ylabel("PCA2")
axes[0].set_title("OPTICS: PCA1 vs PCA2")

# PCA1 vs PCA3
axes[1].scatter(X_pca[:, 0], X_pca[:, 2], c=labels, cmap=cmap, s=10, alpha=0.6)
axes[1].set_xlabel("PCA1")
axes[1].set_ylabel("PCA3")
axes[1].set_title("OPTICS: PCA1 vs PCA3")

# PCA2 vs PCA3
axes[2].scatter(X_pca[:, 1], X_pca[:, 2], c=labels, cmap=cmap, s=10, alpha=0.6)
axes[2].set_xlabel("PCA2")
axes[2].set_ylabel("PCA3")
axes[2].set_title("OPTICS: PCA2 vs PCA3")

# --- Leyenda automática ---
handles, legend_labels = axes[0].get_legend_handles_labels()
legend1 = fig.legend(*axes[0].collections[0].legend_elements(),
                     title="Clusters", loc="upper right")

plt.tight_layout()
plt.show()

In [ ]:
labels_series = pd.Series(labels)
labels_series.value_counts()

In [ ]:
df_clean['Clusters'] = labels

In [ ]:
df_electrones = df_clean[(df_clean['ID'] == 11) | (df_clean['ID'] == -11)]

In [ ]:
df_muones = df_clean[(df_clean['ID'] == 13) | (df_clean['ID'] == -13)]

In [ ]:
len(df_electrones)

In [ ]:
reach = optics.reachability_[optics.ordering_]
plt.figure(figsize=(12,5))
plt.plot(reach)
plt.title("Reachability Plot (OPTICS)")
plt.xlabel("Sample index (ordered)")
plt.ylabel("Reachability distance")
plt.show()

In [ ]:
#Charge_Histogram_All(df_clean,cluster_col='Clusters',bin_width=7)
Charge_Histogram_All(df_clean,df_muones,df_electrones,cluster_col='Clusters',bin_width=2)

In [ ]:
df_muones['Clusters'].value_counts()

In [ ]:
(len(df_muones)-52636)*100/len(df_muones)

In [ ]:
df_electrones['Clusters'].value_counts()

In [ ]:
(len(df_electrones)-15290)*100/len(df_electrones)